In [1]:
import os
import sys
import psycopg2
from sqlalchemy import create_engine
from io import StringIO
import pandas as pd
import csv
up_one =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(up_one)
up_two =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
sys.path.append(up_two)
from minorlab import metapdb
import subprocess
from collections import defaultdict
import datetime
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
conn={}  # A dict of connections
cursor={}    # A dict of cursors
engine={}
def connect_db(db):
    print (f"     Connecting to {db['name']}...")
    try:
        #print (f"""    con_string is { db['con_string'] }""")
        exec (f"""conn[ "{db["name"]}" ] = psycopg2.connect("{db["con_string"]}")""" )
        exec (f"""cursor[ "{db['name']}" ] = conn[ "{db['name']}" ].cursor()""") # Pandas read_sql_query does not need a cursor
        exec (f"""engine["{db['name']}"] = create_engine( "{db['engine_string']}") """)
        print(f"  Connected to { db['name'] }.")
    except:
        print ('Check VPN. Not Connected.   <<<<===========#################')
connect_db(metapdb)
#connect_db(manuscripts)
print ('Connections done.')
# Run this cell to use single database functions
con = conn['metapdb']
cur = cursor['metapdb']
eng = engine['metapdb']

def df_to_empty_table(df, table,db='metapdb'):
    #create table using SLQalchemy 
    parts = table.split('.')
    if len(parts) ==1:
        schema='public'
        table_name=table
    else:
        schema, table_name = parts
    empty_df=  df.loc[df.iloc[:,0]=='Nonsense']
    try:
        empty_df.to_sql( schema=schema,name=table_name, con=engine[db], if_exists = 'replace', index=False)
        print (f'  Empty table {table} created for DataFrame with columns {empty_df.columns.to_list()}')
    except Exception as error:
        print (f'There was a problem with the SQLalchemy part. No table created.\n{error}')
        return 0
    return 1
    
def df_to_table(df, table,db='metapdb'):
    ''' Loads a DataFrame into a table. 
        If table is in the form schema.table, the schema will be used, otherwise schema="public".
        3rd ARG is databse; metapdb is default.'''
    if df_to_empty_table(df,table,db) == 0:
        return 0
    # save dataframe to an in memory buffer
    buffer = StringIO()
    
    df.to_csv(buffer, index=False, header=False,sep=';',quoting=csv.QUOTE_NONE,escapechar='\\')
    buffer.seek(0)
    
    # Try to copy from the buffer to the table
    try:
        print (f"""  Starting copy_from(buffer)""" )
        cursor[db].copy_from(buffer, table, sep=";",)
        conn[db].commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn[db].rollback()
        return 1
    print(f"df_to_table done for {table}.") 

## Slight reformat of gencode information

In [ ]:
with open ('Data/gencode.v43.pc_translations.fa', 'r') as f_in, open ('Data/gencode_v43_translations_mod.fa', 'w') as f_out:
    for line in f_in.readlines():
        if line[0] == ">":
            parts = line.split('|')
            f_out.write(f">{parts[5]}|{parts[1]}\n")
        else:
            f_out.write(line)

### Get list of human pdbs directly from PDB


In [ ]:
human_pdb= pd.read_csv ('Data/PDBe_human_entities.csv')
human_pdb_red = human_pdb.loc[:,['pdb_id','entity_id']]
df_to_table(human_pdb_red, 'temp_dcoop.human_pdb')

In [ ]:
sql = """select ep.pdbid, ep.pdbx_strand_id as chains
            , ep.pdbx_seq_one_letter_code_can  as seq
        from temp_dcoop.human_pdb hp
        join pdbj.entity_poly ep on ep.pdbid = hp.pdb_id
            and hp.entity_id = cast(ep.entity_id as INT)
        where ep.type='polypeptide(L)'
        order by pdbid, pdbx_strand_id"""
df = pd.read_sql (sql,eng)
df
len (df['pdbid'].unique())

## Write out as pdb_human_seqs.fa

In [ ]:
with open ('Data/pdb_human_seqs-230227.fa', 'w') as f:
    for i,parts in df.iterrows():
        chains = chains.replace(',','_')
        f.write(f">{parts['pdbid']}_{parts['chains']}\n{parts['seq']}\n")

## eliminate seqs < 30 AA and group by 100% identity

Group by reduces the list from ~97K to ~39K  
Removing chains less than 30AA removes 3167 


In [ ]:
df2=df.copy()
df2['id']=df2['pdbid']+'_'+ df2['chains']
df2=df2.loc[:,['id','seq']]
df2 = df2.loc[ df2['seq'].str.len() >= 30 ]
print (f'df({len(df)}) - df2({len(df2)}) = {len(df) -  len(df2)} removed by 30 AA cutoff') 
df2.head()

In [ ]:
groups = df2.groupby('seq')['id'].apply(list)
df2_g = groups.reset_index(name='chains')
print (f'df2({len(df2)}) - df2_g({len(df2_g)}) = {len(df2) -  len(df2_g)} less seqs after clustering at 100%') 
df2_g.head()

### get representative chain as for name and write out .fa

In [ ]:
df2_g['rep'] = [i[0] for i in df2_g['chains']]
with open ('Data/pdb_unique_human_seqs_230301.fa', 'w') as f:
    for i,parts in df2_g.iterrows():
        f.write(f">{parts['rep']}\n{parts['seq']}\n")

# Blast part
blastp -db ./a_gencode_fasta_same_prot_clustered.v35.fa -query ./uniprot_reviewed_canonical_and_isoform.fasta  
      -outfmt "6 qseqid qlen qstart qend sseqid slen sstart send evalue bitscore length nident gaps"   
      -out run_UN_to_GC_blast_result.tsv -num_threads 24 >> run_UN_to_GC_blast_result_stdout.txt 2>&1
makeblastdb -in <fasta file> -out <database name> -dbtype <type> -title <title> -parse_seqids
    
__[TEXT](https://www.ncbi.nlm.nih.gov/books/NBK279684/table/appendices.T.options_common_to_all_blast/)__

In [ ]:
blast_path = "/home/daroc/ncbi-blast-2.13.0+/bin"
blastp = os.path.join (blast_path, 'blastp')
makedb = os.path.join (blast_path, 'makeblastdb')
s='gencode_v43_translations_mod.fa'
#q='one_pdb.fa'
#q='ten_percent_pdb.fa'
#q='some_pdbs.fa'
q='pdb_unique_human_seqs_230301.fa'
blast_options="-evalue 1e-20 -max_hsps 2 -num_threads 8"

columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
column_str = " ".join(columns)
s_root = s.replace('.fa','')
outfmt = f"6 {column_str}"


In [ ]:
makeblastdb ('gencode_v43_translations_mod.fa')

In [ ]:
#Project specific functions
# Make blast dbs
def makeblastdb (fasta):
    #  Note : This was problematic on OneDrive 
    root = fasta.replace('.fa','')
    cmd = f"""{makedb} -in {fasta} -out {root} -dbtype prot -title "Blastdb for {fasta}" -parse_seqids """
    print (cmd)
    db=subprocess.getoutput(cmd)
    if db[0:5]=="USAGE":
        print (f"Something went wrong with creation of the database\n{db}")

##### Make Databases in some cases

NOTE:  This will not work from OneDrive directory becuase of the spaces in the name. 

In [ ]:
#  Should not need to edit below here 
###
makeblastdb(q)



### Run blast with tabular output

In [ ]:
# DF output
start_time = datetime.datetime.now()
print (f'Starting at {start_time}')
blast_cmd = f'{blastp} -query "Data/{q}" -db "Data/Blast_databases/{s_root}" {blast_options} -outfmt "{outfmt}"'
print (f"{blast_cmd}\n")
blast_results=subprocess.getoutput(blast_cmd)
results_list = [line.split('\t') for line in blast_results.splitlines()]
blast_df = pd.DataFrame(results_list, columns = columns)
end_time =  datetime.datetime.now()
print ( f'It is now {end_time}. Blast took { end_time - start_time}')
try:
    blast_df.to_excel('Data/pdb_vs_gencode.xlsx')
except:
    print ("\n\nCouldn't write Excel file!!!")
blast_df


### Write out full blast results  
Note: it is too large to fit in one Excel spreadsheet. Write out as tsv file and 2 partial 

In [ ]:
blast_df.to_csv('Data/blast_fullPDB_vs_gencode.tsv', sep='\t')

In [ ]:
blast_df.iloc[0:1000000].to_excel('Data/pdb_vs_gencode-part1.xlsx')
blast_df.iloc[1000001:].to_excel('Data/pdb_vs_gencode-part2.xlsx')

### Run blast and see full results

In [ ]:
# For full results including alignments
blast_cmd = f'{blastp} -query "Data/{q}" -subject "Data/{s}"  {blast_options}'
print (f"{blast_cmd}\n")
blast_results=subprocess.getoutput(blast_cmd)
print (blast_results)

### The function blast_one will extract one PDB seq and run blast on it. 
USE: blast_one(pdb, [align=False] )
The default will give tabular result. align=True will give alignments



In [ ]:
blast_df = blast_one('3iej_A,B')
blast_df

In [ ]:
blast_df.to_excel('Data/3iej.xlsx')

### Function to reduce the data to top matches of each PDB
Function is called top_five, the the second option is the numner of hits to include

In [ ]:
def top_five(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            keep.append(True)
        else:
            count += 1
            if count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
#top_five_df = top_five(copy_df)
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')
        

In [ ]:
top_2_df = top_five(blast_df, 2)
top_2_df.to_csv('Data/top2_pdb_vs_gencode.tsv', sep='\t')


In [ ]:
top_five_df.to_excel("Data/top_five.xlsx")

## Identify genes with more than one isoform. 

In [ ]:
df_to_check = top_2_df.copy()
# Build a dictionary where the values are a set of isoforms for each gene
genes=defaultdict(set)
for id in df_to_check['sseqid']:
    try:
        id = id.split('|')[0]
        parts = id.split('-')
        gene='-'.join(parts[0:-1])
        iso=parts[-1]
        #print (id, gene, iso)
        genes[gene].add(iso)
    except:
        pass
# Create a list (check) of genes that have more than 1 isoform represented.
check = []
for g in genes:
    if len(genes[g]) > 1:
        #print (g,genes[g])
        check.append(g)
print (f'check is {len(check)} long')

# Create list of df indicies of all genes that have more than one df 
found=[]
for i,row in df_to_check.iterrows():
    #print (row['sseqid'].split('-')[0])
    try:
        if row['sseqid'].split('-')[0] in check:
            found.append(i)
    except:
        pass
#print (f'found is {found}')
found_df = df_to_check.loc[found]
found_df.sort_values(by=['sseqid'], inplace=True)
found_df



In [ ]:
found_df.to_excel('Data/found_isos_from_2.xlsx')


### Function to extract sequences of Isoforms and related sequences
USE: get_gene_seqs (gene, df=found_df)  
By default, will extract data from found_df, but can be specified 

In [2]:
pdb_file='Data/pdb_unique_human_seqs_230301.fa'
gene_file='Data/gencode_v43_translations_mod.fa'  
def extract_sequence(filename, sequence_id):
    sequence = ""
    with open(filename, "r") as f:
        for line in f:
            if line.startswith(">") and sequence_id in line:
                sequence += line
                # Found the sequence ID we want
                while True:
                    line = f.readline().strip()
                    if line.startswith(">") or not line:
                        # Reached the end of the sequence
                        break
                    sequence += line
    sequence += "\n"
    return sequence
def get_gene_seqs (gene, df=found_df):
    matches = found_df.loc[ found_df['sseqid'].str.contains(gene) ]
    isos = set()
    pdbs = set()
    for i,row in matches.iterrows():
        isos.add(row['sseqid'])
        pdbs.add(row['qseqid'])
    seqs=""
    for pdb in pdbs:
        seqs+= extract_sequence (pdb_file, pdb)
    for iso in isos:
        seqs+= extract_sequence (gene_file, iso)
    return (seqs)

s=get_gene_seqs('SF3B1')
print (s)

NameError: name 'found_df' is not defined

### Create a directory with sequence files for all the matches

In [ ]:
# For each gene in found_df, create a FASTA file with PDB and isoform sequences. 
# Also creates a list (done) of the gene names
done = []
for id in found_df['sseqid']:
    try:
        id = id.split('|')[0]
        parts = id.split('-')
        gene='-'.join(parts[0:-1])
        if gene not in done:
            done.append(gene)
            s = get_gene_seqs(gene)
            with open (f'Data/Seq_groups/{gene}.fa', 'w') as out:
                out.write(s)
    except:
        pass
    
    

In [ ]:
tdf= pd.DataFrame(done)
tdf.to_excel('gene_notes.xlsx')

In [ ]:
last = "ABCB8"

In [ ]:
fresh=0
for g in tdf[0]:
    print (g)
    if fresh==0 and g!=last:
        continue
    fresh=1
    last = g
    cmd = f'clustalo -i Data/Seq_groups/{g}.fa  --outfmt=clu -o Data/Seq_groups/{g}.msa'
    print (cmd)
    r=subprocess.getoutput(cmd)
    print (r)
    with open (f"Data/Seq_groups/{g}.msa") as f:
        rr=f.read()
        print (rr)
        
    read = input ("Ready to move on? f to see sequence file ")
    if read.lower() == 'f':
        with open (f'Data/Seq_groups/{g}.fa') as seqs:
            print (seqs.read())
        read = input ("Ready to move on? f to see sequence file ")